In [8]:
import pandas as pd
import numpy as np 
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [2]:
def target_search(target_name):
    target = new_client.target
    res = target.filter(pref_name__icontains = target_name)
    if len(res) > 200:
        raise ValueError('Search Target too broad, Try Another Search Term.')
    else:
        targets = pd.DataFrame(res)
        return(targets) #Returns a df of targets matching your search term
    

In [4]:
df = target_search("carbonic")
df.head()

,cross_references,organism,pref_name,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P00918', 'xref_name': None, 'xre...",Homo sapiens,Carbonic anhydrase II,False,CHEMBL205,"[{'accession': 'P00918', 'component_descriptio...",SINGLE PROTEIN,9606
1,"[{'xref_id': 'P07451', 'xref_name': None, 'xre...",Homo sapiens,Carbonic anhydrase III,False,CHEMBL2885,"[{'accession': 'P07451', 'component_descriptio...",SINGLE PROTEIN,9606
2,"[{'xref_id': 'P23280', 'xref_name': None, 'xre...",Homo sapiens,Carbonic anhydrase VI,False,CHEMBL3025,"[{'accession': 'P23280', 'component_descriptio...",SINGLE PROTEIN,9606
3,"[{'xref_id': 'P00915', 'xref_name': None, 'xre...",Homo sapiens,Carbonic anhydrase I,False,CHEMBL261,"[{'accession': 'P00915', 'component_descriptio...",SINGLE PROTEIN,9606
4,"[{'xref_id': 'P27139', 'xref_name': None, 'xre...",Rattus norvegicus,Carbonic anhydrase II,False,CHEMBL4706,"[{'accession': 'P27139', 'component_descriptio...",SINGLE PROTEIN,10116


In [5]:
def get_bioactivities_data(target_chembl_ID):
    activity = new_client.activity
    res = activity.filter(target_chembl_id=target_chembl_ID).filter(standard_type="IC50").only(['molecule_chembl_id','canonical_smiles','standard_value', 'standard_type'])
    df = pd.DataFrame(res[0:500])#Set a limit for now, take too long to compute over 500 activities
    df['standard_value'] = df['standard_value'].astype(float)
    
    conditions = [
        (df['standard_value'] >= 10000),
        (df['standard_value'] <= 1000)
    ]
    choices = ['inactive', 'active']
    df['bioactivity_class'] = np.select(conditions, choices, default = 'intermediate')
    return(df)

In [7]:
df_2 = get_bioactivities_data('CHEMBL3927')
df_2.head()

,canonical_smiles,molecule_chembl_id,standard_type,standard_value,type,value,bioactivity_class
0,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,CHEMBL187579,IC50,7200.0,IC50,7.2,intermediate
1,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,CHEMBL188487,IC50,9400.0,IC50,9.4,intermediate
2,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,CHEMBL185698,IC50,13500.0,IC50,13.5,inactive
3,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,CHEMBL426082,IC50,13110.0,IC50,13.11,inactive
4,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],CHEMBL187717,IC50,2000.0,IC50,2.0,intermediate


## Calculate Lipinski descriptors
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

1. Molecular weight < 500 Dalton
2. Octanol-water partition coefficient (LogP) < 5
3. Hydrogen bond donors < 5
4. Hydrogen bond acceptors < 10

In [9]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [10]:
df_lipinski = lipinski(df_2.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,281.271,1.89262,0.0,5.0
1,415.589,3.81320,0.0,2.0
2,421.190,2.66050,0.0,4.0
3,293.347,3.63080,0.0,3.0
4,338.344,3.53900,0.0,5.0
...,...,...,...,...
128,338.359,3.40102,0.0,5.0
129,296.366,3.44330,0.0,3.0
130,276.291,4.09564,0.0,3.0
131,278.307,3.29102,0.0,3.0


In [15]:
df_combined = pd.concat([df_2, df_lipinski], axis = 1)
df_combined

,canonical_smiles,molecule_chembl_id,standard_type,standard_value,type,value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,CHEMBL187579,IC50,7200.0,IC50,7.2,intermediate,281.271,1.89262,0.0,5.0
1,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,CHEMBL188487,IC50,9400.0,IC50,9.4,intermediate,415.589,3.81320,0.0,2.0
2,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,CHEMBL185698,IC50,13500.0,IC50,13.5,inactive,421.190,2.66050,0.0,4.0
3,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,CHEMBL426082,IC50,13110.0,IC50,13.11,inactive,293.347,3.63080,0.0,3.0
4,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],CHEMBL187717,IC50,2000.0,IC50,2.0,intermediate,338.344,3.53900,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
128,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,CHEMBL2146517,IC50,10600.0,IC50,10.6,inactive,338.359,3.40102,0.0,5.0
129,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,CHEMBL187460,IC50,10100.0,IC50,10.1,inactive,296.366,3.44330,0.0,3.0
130,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,CHEMBL363535,IC50,11500.0,IC50,11.5,inactive,276.291,4.09564,0.0,3.0
131,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,CHEMBL227075,IC50,10700.0,IC50,10.7,inactive,278.307,3.29102,0.0,3.0


## Convert IC50 to pIC50
To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).

This custom function pIC50() will accept a DataFrame as input and will:

Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10
Take the molar value and apply -log10
Delete the standard_value column and create a new pIC50 column

In [16]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [17]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [18]:
df_norm = norm_value(df_combined)
df_norm

c:\Users\jacks\anaconda3\envs\drug_discovery\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,canonical_smiles,molecule_chembl_id,standard_type,type,value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,CHEMBL187579,IC50,IC50,7.2,intermediate,281.271,1.89262,0.0,5.0,7200.0
1,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,CHEMBL188487,IC50,IC50,9.4,intermediate,415.589,3.81320,0.0,2.0,9400.0
2,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,CHEMBL185698,IC50,IC50,13.5,inactive,421.190,2.66050,0.0,4.0,13500.0
3,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,CHEMBL426082,IC50,IC50,13.11,inactive,293.347,3.63080,0.0,3.0,13110.0
4,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],CHEMBL187717,IC50,IC50,2.0,intermediate,338.344,3.53900,0.0,5.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
128,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,CHEMBL2146517,IC50,IC50,10.6,inactive,338.359,3.40102,0.0,5.0,10600.0
129,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,CHEMBL187460,IC50,IC50,10.1,inactive,296.366,3.44330,0.0,3.0,10100.0
130,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,CHEMBL363535,IC50,IC50,11.5,inactive,276.291,4.09564,0.0,3.0,11500.0
131,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,CHEMBL227075,IC50,IC50,10.7,inactive,278.307,3.29102,0.0,3.0,10700.0


In [19]:
df_final = pIC50(df_norm)
df_final

c:\Users\jacks\anaconda3\envs\drug_discovery\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


,canonical_smiles,molecule_chembl_id,standard_type,type,value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,CHEMBL187579,IC50,IC50,7.2,intermediate,281.271,1.89262,0.0,5.0,5.142668
1,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,CHEMBL188487,IC50,IC50,9.4,intermediate,415.589,3.81320,0.0,2.0,5.026872
2,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,CHEMBL185698,IC50,IC50,13.5,inactive,421.190,2.66050,0.0,4.0,4.869666
3,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,CHEMBL426082,IC50,IC50,13.11,inactive,293.347,3.63080,0.0,3.0,4.882397
4,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],CHEMBL187717,IC50,IC50,2.0,intermediate,338.344,3.53900,0.0,5.0,5.698970
...,...,...,...,...,...,...,...,...,...,...,...
128,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,CHEMBL2146517,IC50,IC50,10.6,inactive,338.359,3.40102,0.0,5.0,4.974694
129,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,CHEMBL187460,IC50,IC50,10.1,inactive,296.366,3.44330,0.0,3.0,4.995679
130,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,CHEMBL363535,IC50,IC50,11.5,inactive,276.291,4.09564,0.0,3.0,4.939302
131,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,CHEMBL227075,IC50,IC50,10.7,inactive,278.307,3.29102,0.0,3.0,4.970616
